In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [5]:
nums=np.arange(1,51)
rdd1=sc.parallelize(nums)
rdd1.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

Find the sum, average, maximum, minimum, and count

In [6]:
#Sum
sum=rdd1.sum()
print(sum)

1275


In [7]:
#avg
avg=sum/rdd1.count()
print(avg)

25.5


In [8]:
#Count
rdd1.count()

50

In [9]:
#Min
rdd1.min()

1

In [10]:
#Max
rdd1.max()

50

Count how many numbers are even vs. odd.

In [11]:
even_count=rdd1.filter(lambda X: X % 2 ==0).count()

odd_count =rdd1.filter( lambda x : x % 2 != 0).count()

print('even count is ',even_count)
print('odd ount is ',odd_count)

even count is  25
odd ount is  25


You have the following data of people info ('Name', 'Age'), answer the following questions

In [12]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [13]:
oldest_person= rdd_people.max(key=lambda x:x [1])

print('oldset is',oldest_person)

oldset is ('Khaled', 40)


Compute the average age

In [14]:
ages_rdd=rdd_people.map(lambda x:x[1])
total_age=ages_rdd.sum()
count_people=ages_rdd.count()
avg_age = total_age / count_people 
print('avg age :',avg_age)

avg age : 31.666666666666668


Group all the names by their age

In [15]:
rdd_by_age = rdd_people.map(lambda x: (x[1], x))
grouped = rdd_by_age.groupByKey().mapValues(list)


result = grouped.sortByKey()

print(result.collect())

[(25, [('Nada ', 25), ('Nada ', 25)]), (30, [('Mona', 30)]), (35, [('Ahmed', 35), ('Ahmed', 35)]), (40, [('Khaled', 40)])]


Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [16]:
spark


In [17]:
rdd_russia = sc.textFile("/data/russia.txt")
rdd_russia.collect()

["['Russia is the largest country in the world by land area',",
 " 'Moscow is the capital city of Russia',",
 " 'The Russian language is one of the most widely spoken languages in the world',",
 " 'Russia is known for its rich history and culture',",
 " 'The Trans-Siberian Railway is the longest railway line in the world',",
 " 'Russia has a strong tradition in literature, music and ballet',",
 " 'The country is famous for its cold winters and vast landscapes',",
 " 'Russia is a major player in global energy production']"]

Count the total number of lines.

In [20]:
line_count=rdd_russia.count()
print('no of line:',line_count)

no of line: 8


Count how many lines contain the word "Russia"

In [21]:
russia_line=rdd_russia.filter(lambda line: "Russia" in line ).count()
print('no :',russia_line)

no : 6


Find the most 5 frequent word in the file.

In [25]:
words_rdd=rdd_russia.flatMap(lambda line: line.split())
word_ones=words_rdd.map(lambda word : (word,1))
word_counts=word_ones.reduceByKey(lambda a,b : a + b)

sorted_words=word_counts.sortBy(lambda x: x [1],ascending=False)
TOP5=sorted_words.take(5)
print('top5:',TOP5)


top5: [('is', 7), ('the', 7), ('in', 5), ("'The", 3), ('and', 3)]


Tokenize words

In [27]:
import re

tokens_rdd = rdd_russia.flatMap(
    lambda line: re.findall(r"\b\w+\b", line.lower())
)


print(tokens_rdd.take(20))

['russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'moscow', 'is', 'the', 'capital', 'city', 'of', 'russia', 'the', 'russian']


Remove stopwords (a, the, is, to, in, of). 

In [32]:
stopwords={'a','the','is','to','in','of'}

filter_stop=tokens_rdd.filter(lambda word : word not in stopwords)

print(filter_stop.take(20))

['russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'moscow', 'capital', 'city', 'russia', 'russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'russia']


Count the frequency of each word

In [33]:
print(word_counts.take(20))

[("['Russia", 1), ('largest', 1), ('country', 2), ('world', 1), ('by', 1), ('land', 1), ("area',", 1), ('capital', 1), ('of', 2), ("Russia',", 1), ("'The", 3), ('language', 1), ('most', 1), ('widely', 1), ("world',", 2), ('known', 1), ('for', 2), ('history', 1), ('and', 3), ("culture',", 1)]


In [18]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [35]:
df.printSchema()


df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [36]:
df.select("name","salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [37]:
from pyspark.sql import functions as F

df.select(F.avg('salary').alias("avg_salary")).show()

+----------+
|avg_salary|
+----------+
|    6000.0|
+----------+



Filter employees older than 28

In [38]:
df.filter(df.age>28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [39]:
df.select(F.countDistinct("name").alias("distinct_names")).show()

+--------------+
|distinct_names|
+--------------+
|             4|
+--------------+



Group by a the name column and find average salary

In [40]:
df.groupBy("name").agg(F.avg("salary").alias("avg_salary")).show()

+------+----------+
|  name|avg_salary|
+------+----------+
|   Ali|    4000.0|
|Mariam|    6750.0|
|  Omar|    6750.0|
|  Sara|    5000.0|
+------+----------+



In [53]:
df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show(5)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|PK \b\b\b �P9[               xl/drawings/drawing1.xml��]n�0\f\a��U�iZC^�N0�%n�����~�J6i{m�?���nt��Db|#�z��]#�o��(8��`��F\��n��5�ϼ�"�{^��}��ZJV=:�2\f�Ӵ|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                       � ��:�\t�IvVΫ�E�@...|
|                                                                                                                                                       �A��8!�b��f଩\a...|
|                                                                                                                                 

Find the avg sales 

In [54]:
df1.select(F.avg("Sales").alias("avg_sale")).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Sales` cannot be resolved. Did you mean one of the following? [`PK  �P9[               xl/drawings/drawing1.xml��]n�0��U�iZC^�N0�%n�����~�J6i{m�?���nt��Db|#�z��]#�o��(8��``��F\��n��5�ϼ�"�{^��}��ZJV=:�2�Ӵ`].;
'Aggregate [avg('Sales) AS avg_sale#240]
+- Relation [PK  �P9[               xl/drawings/drawing1.xml��]n�0��U�iZC^�N0�%n�����~�J6i{m�?���nt��Db|#�z��]#�o��(8��`��F\��n��5�ϼ�"�{^��}��ZJV=:�2�Ӵ#230] csv


Replace null name with 'Unknown' and sales with the avg sales of the column 

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

